In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
#Извлечем данные о регистрации и посещениях👀
dfReg_time = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-v-ananin/shared/problem1-reg_data.csv', sep = ';')
dfReg_time

,reg_ts,uid
0,911382223,1
1,932683089,2
2,947802447,3
3,959523541,4
4,969103313,5
...,...,...
999995,1600874034,1110618
999996,1600874086,1110619
999997,1600874139,1110620
999998,1600874191,1110621


In [4]:
dfAuth_time = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-v-ananin/shared/problem1-auth_data.csv', sep = ';')
dfAuth_time

,auth_ts,uid
0,911382223,1
1,932683089,2
2,932921206,2
3,933393015,2
4,933875379,2
...,...,...
9601008,1600874034,1110618
9601009,1600874086,1110619
9601010,1600874139,1110620
9601011,1600874191,1110621


In [5]:
dfAuth_time.info()
dfAuth_time.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9601013 entries, 0 to 9601012
Data columns (total 2 columns):
 #   Column   Dtype
---  ------   -----
 0   auth_ts  int64
 1   uid      int64
dtypes: int64(2)
memory usage: 146.5 MB


auth_ts    9180915
uid        1000000
dtype: int64

In [6]:
dfReg_time.info()
dfReg_time.nunique()
#Посмотрел на датасеты и информацию о них. Время регистрации и входов следует перевести в datetime🤔 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype
---  ------  --------------    -----
 0   reg_ts  1000000 non-null  int64
 1   uid     1000000 non-null  int64
dtypes: int64(2)
memory usage: 15.3 MB


reg_ts    1000000
uid       1000000
dtype: int64

In [23]:
def retention(dfReg_time, dfAuth_time):
    #Сначала сджоиним данные о времени регистрации и входа пользователей
    df_join = pd.merge(dfReg_time, dfAuth_time, on='uid', suffixes=('_reg', '_auth'))

    #Преобразование дат в тип datetime из Unix'a😤
    df_join['reg_ts'] = pd.to_datetime(df_join['reg_ts'], unit='s')
    df_join['auth_ts'] = pd.to_datetime(df_join['auth_ts'], unit='s')

    #Преобразуем datetime просто в дату в обоих столбцах
    df_join['reg_date'] = df_join['reg_ts'].dt.date
    df_join['auth_date'] = df_join['auth_ts'].dt.date

    #Группировка по дню регистрации и дню захода в игру и подсчет уникальных пользователей для комбинированной группировки
    group = df_join.groupby(['reg_date', 'auth_date'])['uid'].nunique().reset_index()

    #Соотнесем и подсчитаем кол-во уникальных пользователей в день регистрации 
    #(т.е. те, кто зарегистрировался и вошел в тот же день)
    total_reg_users = group[group['reg_date'] == group['auth_date']]['uid'].sum()

    #Рассчет retention с учетом любой комбинации даты возвращения😵
    group['retention'] = group['uid'] / total_reg_users

    return group

#Запишем результат функции (а это новый датафрейм group) в новую переменную и обратимся к ней за результатом🎉
result = retention(dfReg_time, dfAuth_time)

result
#Ответ :👆

,reg_date,auth_date,uid,retention
0,1998-11-18,1998-11-18,1,0.000001
1,1999-07-22,1999-07-22,1,0.000001
2,1999-07-22,1999-07-25,1,0.000001
3,1999-07-22,1999-07-31,1,0.000001
4,1999-07-22,1999-08-05,1,0.000001
...,...,...,...,...
2716223,2020-09-21,2020-09-22,31,0.000031
2716224,2020-09-21,2020-09-23,49,0.000049
2716225,2020-09-22,2020-09-22,1641,0.001641
2716226,2020-09-22,2020-09-23,14,0.000014


In [6]:
#Решаем второй вопрос. Для начала взгляну на данные
dfTask2 = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-v-ananin/Final_project/Проект_1_Задание_2.csv', sep = ';')
dfTask2

,user_id,revenue,testgroup
0,1,0,b
1,2,0,a
2,3,0,a
3,4,0,b
4,5,0,b
...,...,...,...
404765,404766,0,a
404766,404767,0,b
404767,404768,231,a
404768,404769,0,a


In [9]:
#Оценим данные
dfTask2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404770 entries, 0 to 404769
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    404770 non-null  int64 
 1   revenue    404770 non-null  int64 
 2   testgroup  404770 non-null  object
dtypes: int64(2), object(1)
memory usage: 9.3+ MB


In [10]:
#Группировка по группе) и аггрегация 
df2 = dfTask2.groupby('testgroup')['revenue'].agg(['count', 'sum'])

#Подсчет количества клиентов с ненулевым значением в столбце revenue
clients = dfTask2[dfTask2['revenue'] != 0]['user_id'].nunique()

#clients_A = dfTask2[(dfTask2['revenue'] != 0) & (dfTask2['testgroup'] == 'a')]['user_id'].nunique()
#В переменной clients_A сумма уникальных клиентов 1928 как в задании — значит двигаемся правильно!

#И тут же мы выяснили, что группа А у нас контрольная, а В тестовая 😉

In [11]:
#Рассчет среднего чека
df2['avg_check'] = df2['sum'] / df2['count']

#Рассчет повторных покупок
df2['rep_purchases'] = df2['count'] / dfTask2['user_id'].nunique()

#Рассчет ARPPU
df2['ARPPU'] = df2['rep_purchases'] * df2['avg_check']

#Рассчет конверсии
df2['conversion'] = clients / dfTask2['user_id'].nunique()

#Рассчет ARPU
df2['ARPU'] = df2['conversion'] * df2['ARPPU']


df2

,count,sum,avg_check,rep_purchases,ARPPU,conversion,ARPU
testgroup,,,,,,,
a,202103,5136189,25.413720,0.499303,12.689154,0.009223,0.117026
b,202667,5421603,26.751287,0.500697,13.394281,0.009223,0.123529


In [12]:
#Имеются результаты A/B теста, в котором двум группам пользователей предлагались различные наборы акционных предложений. 
#Известно, что ARPU в тестовой группе выше на 5%, чем в контрольной. 
#При этом в контрольной группе 1928 игроков из 202103 оказались платящими, а в тестовой – 1805 из 202667.
#Какой набор предложений можно считать лучшим? 
#Какие метрики стоит проанализировать для принятия правильного решения и как?

#ОТВЕТ:
#🤓 На основании высчитанных ключевых (на мой взгляд) метрик (представленный в итоговом датафрейме df2),
#можно предположить, что тестовая группа В справилась лучше, даже при условии, что выборка больше, а платящих меньше.
#Вероятно стоит выкатывать набор акций на большую аудиторию.